
# Introduction

In this notebook we will cluster and project research article embeddings. The end result is a model which we can use to visualize different clusters in the article space. 

In [0]:
df = spark.read.table("main.default.ARXIV")
sample_fraction = 0.1
df = df.sample(sample_fraction)
display(df.count())

238300

In [0]:
display(df.limit(3))

abstract,authors_parsed,categories,id,submitter,title,update_date,year
"Imitation learning holds great promise for addressing the complex task of autonomous urban driving, as experienced human drivers can navigate highly challenging scenarios with ease. While behavior cloning is a widely used imitation learning approach in autonomous driving due to its exemption from risky online interactions, it suffers from the covariate shift issue. To address this limitation, we propose a context-conditioned imitation learning approach that employs a policy to map the context state into the ego vehicle's future trajectory, rather than relying on the traditional formulation of both ego and context states to predict the ego action. Additionally, to reduce the implicit ego information in the coordinate system, we design an ego-perturbed goal-oriented coordinate system. The origin of this coordinate system is the ego vehicle's position plus a zero mean Gaussian perturbation, and the x-axis direction points towards its goal position. Our experiments on the real-world large-scale Lyft and nuPlan datasets show that our method significantly outperforms state-of-the-art approaches.","List(List(Guo, Ke, ), List(Jing, Wei, ), List(Chen, Junbo, ), List(Pan, Jia, ))",cs.RO,2305.02649,Ke Guo,CCIL: Context-conditioned imitation learning for urban driving,2023-05-05,2023
"Controllable image captioning is an emerging multimodal topic that aims to describe the image with natural language following human purpose, $\textit{e.g.}$, looking at the specified regions or telling in a particular text style. State-of-the-art methods are trained on annotated pairs of input controls and output captions. However, the scarcity of such well-annotated multimodal data largely limits their usability and scalability for interactive AI systems. Leveraging unimodal instruction-following foundation models is a promising alternative that benefits from broader sources of data. In this paper, we present Caption AnyThing (CAT), a foundation model augmented image captioning framework supporting a wide range of multimodel controls: 1) visual controls, including points, boxes, and trajectories; 2) language controls, such as sentiment, length, language, and factuality. Powered by Segment Anything Model (SAM) and ChatGPT, we unify the visual and language prompts into a modularized framework, enabling the flexible combination between different controls. Extensive case studies demonstrate the user intention alignment capabilities of our framework, shedding light on effective user interaction modeling in vision-language applications. Our code is publicly available at https://github.com/ttengwang/Caption-Anything.","List(List(Wang, Teng, ), List(Zhang, Jinrui, ), List(Fei, Junjie, ), List(Zheng, Hao, ), List(Tang, Yunlong, ), List(Li, Zhe, ), List(Gao, Mingqi, ), List(Zhao, Shanshan, ))",cs.CV,2305.02677,Teng Wang,Caption Anything: Interactive Image Description with Diverse Multimodal Controls,2023-07-07,2023
"In this paper, we study row graphs of Toeplitz matrices. The notion of row graphs was introduced by Greenberg et al. in 1984 and is closely related to the notion of competition graphs, which has been extensively studied since Cohen had introduced it in 1968. To understand the structure of the row graphs of Toeplitz matrices, which seem to be quite complicated, we have begun with Toeplitz matrices whose row graphs are triangle-free. We could show that if the row graph G of a Toeplitz matrix T is triangle-free, then T has the maximum row sum at most 2. Furthermore, it turns out that G is a disjoint union of paths and cycles whose lengths cannot vary that much in such a case. Then we study (0, 1)-Toeplitz matrices whose row graphs have only path components, only cycle components, and a cycle component of specific length, respectively. In particular, we completely characterize a (0, 1)-Toeplitz matrix whose row graph is a cycle.","List(List(Cheon, Gi-Sang, ), List(Kang, Bumtle, ), List(Ki


# Cleaning and tokenization

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer
from pyspark.sql.functions import udf, col, regexp_replace

In [0]:
def clean_text(df, input_column = "abstract", output_column = "abstract_clean"):
    # Remove LaTeX math equations
    df = df.withColumn(output_column, regexp_replace(input_column, r"(\$+)(?:(?!\1)[\s\S])*\1", ""))
    df = df.withColumn(output_column, regexp_replace(output_column, r"[\d-]", ""))
    df = df.withColumn(output_column, regexp_replace(output_column, r"[()£^.,\\]+", ""))
    return df

In [0]:
# Remove LaTeX math equations
df = clean_text(df, output_column = "abstract_math_free")
display(df.limit(3))

abstract,authors_parsed,categories,id,submitter,title,update_date,year,abstract_math_free
"Imitation learning holds great promise for addressing the complex task of autonomous urban driving, as experienced human drivers can navigate highly challenging scenarios with ease. While behavior cloning is a widely used imitation learning approach in autonomous driving due to its exemption from risky online interactions, it suffers from the covariate shift issue. To address this limitation, we propose a context-conditioned imitation learning approach that employs a policy to map the context state into the ego vehicle's future trajectory, rather than relying on the traditional formulation of both ego and context states to predict the ego action. Additionally, to reduce the implicit ego information in the coordinate system, we design an ego-perturbed goal-oriented coordinate system. The origin of this coordinate system is the ego vehicle's position plus a zero mean Gaussian perturbation, and the x-axis direction points towards its goal position. Our experiments on the real-world large-scale Lyft and nuPlan datasets show that our method significantly outperforms state-of-the-art approaches.","List(List(Guo, Ke, ), List(Jing, Wei, ), List(Chen, Junbo, ), List(Pan, Jia, ))",cs.RO,2305.02649,Ke Guo,CCIL: Context-conditioned imitation learning for urban driving,2023-05-05,2023,Imitation learning holds great promise for addressing the complex task of autonomous urban driving as experienced human drivers can navigate highly challenging scenarios with ease While behavior cloning is a widely used imitation learning approach in autonomous driving due to its exemption from risky online interactions it suffers from the covariate shift issue To address this limitation we propose a contextconditioned imitation learning approach that employs a policy to map the context state into the ego vehicle's future trajectory rather than relying on the traditional formulation of both ego and context states to predict the ego action Additionally to reduce the implicit ego information in the coordinate system we design an egoperturbed goaloriented coordinate system The origin of this coordinate system is the ego vehicle's position plus a zero mean Gaussian perturbation and the xaxis direction points towards its goal position Our experiments on the realworld largescale Lyft and nuPlan datasets show that our method significantly outperforms stateoftheart approaches
"Controllable image captioning is an emerging multimodal topic that aims to describe the image with natural language following human purpose, $\textit{e.g.}$, looking at the specified regions or telling in a particular text style. State-of-the-art methods are trained on annotated pairs of input controls and output captions. However, the scarcity of such well-annotated multimodal data largely limits their usability and scalability for interactive AI systems. Leveraging unimodal instruction-following foundation models is a promising alternative that benefits from broader sources of data. In this paper, we present Caption AnyThing (CAT), a foundation model augmented image captioning framework supporting a wide range of multimodel controls: 1) visual controls, including points, boxes, and trajectories; 2) language controls, such as sentiment, length, language, and factuality. Powered by Segment Anything Model (SAM) and ChatGPT, we unify the visual and language prompts into a modularized framework, enabling the flexible combination between different controls. Extensive case studies demonstrate the user intention alignment capabilities of our framework, shedding light on effective user interaction modeling in vision-language applications. Our code is publicly available at https://github.com/ttengwang/Caption-Anything.","List(List(Wang, Teng, ), List(Zhang, Jinrui, ), List(Fei, Junjie, ), List(Zheng, Hao, ), List(Tang, Yunlong, ), List(Li, Zhe, ), List(Gao, Mingqi, ), List(Zhao, Shanshan, ))",cs.CV,2305.02677,Ten

In [0]:
word_tokenizer = RegexTokenizer(inputCol = "abstract_math_free", outputCol = "word_tokens", minTokenLength = 3)
stop_words_remover = StopWordsRemover(inputCol = "word_tokens", outputCol = "clean_tokens")

In [0]:
df = word_tokenizer.transform(df)
df = stop_words_remover.transform(df)

In [0]:
display(df.select('id', 'clean_tokens').limit(12))

id,clean_tokens
2305.02649,"List(imitation, learning, holds, great, promise, addressing, complex, task, autonomous, urban, driving, experienced, human, drivers, navigate, highly, challenging, scenarios, ease, behavior, cloning, widely, used, imitation, learning, approach, autonomous, driving, due, exemption, risky, online, interactions, suffers, covariate, shift, issue, address, limitation, propose, contextconditioned, imitation, learning, approach, employs, policy, map, context, state, ego, vehicle's, future, trajectory, rather, relying, traditional, formulation, ego, context, states, predict, ego, action, additionally, reduce, implicit, ego, information, coordinate, system, design, egoperturbed, goaloriented, coordinate, system, origin, coordinate, system, ego, vehicle's, position, plus, zero, mean, gaussian, perturbation, xaxis, direction, points, towards, goal, position, experiments, realworld, largescale, lyft, nuplan, datasets, show, method, significantly, outperforms, stateoftheart, approaches)"
2305.02677,"List(controllable, image, captioning, emerging, multimodal, topic, aims, describe, image, natural, language, following, human, purpose, looking, specified, regions, telling, particular, text, style, stateoftheart, methods, trained, annotated, pairs, input, controls, output, captions, however, scarcity, wellannotated, multimodal, data, largely, limits, usability, scalability, interactive, systems, leveraging, unimodal, instructionfollowing, foundation, models, promising, alternative, benefits, broader, sources, data, paper, present, caption, anything, cat, foundation, model, augmented, image, captioning, framework, supporting, wide, range, multimodel, controls:, visual, controls, including, points, boxes, trajectories;, language, controls, sentiment, length, language, factuality, powered, segment, anything, model, sam, chatgpt, unify, visual, language, prompts, modularized, framework, enabling, flexible, combination, different, controls, extensive, case, studies, demonstrate, user, intention, alignment, capabilities, framework, shedding, light, effective, user, interaction, modeling, visionlanguage, applications, code, publicly, available, https://githubcom/ttengwang/captionanything)"
2305.02690,"List(paper, study, row, graphs, toeplitz, matrices, notion, row, graphs, introduced, greenberg, closely, related, notion, competition, graphs, extensively, studied, since, cohen, introduced, understand, structure, row, graphs, toeplitz, matrices, seem, quite, complicated, begun, toeplitz, matrices, whose, row, graphs, trianglefree, show, row, graph, toeplitz, matrix, trianglefree, maximum, row, sum, furthermore, turns, disjoint, union, paths, cycles, whose, lengths, vary, much, case, study, toeplitz, matrices, whose, row, graphs, path, components, cycle, components, cycle, component, specific, length, respectively, particular, completely, characterize, toeplitz, matrix, whose, row, graph, cycle)"
2305.02704,"List(fractional, programming, plays, crucial, role, wireless, network, design, many, relevant, problems, involve, maximizing, minimizing, ratio, terms, notice, maximization, case, minimization, case, converted, general, dealt, separately, previous, studies, thus, existing, method, maximizing, ratios, typically, work, minimization, case, vice, versa, however, objective, mixed, maxandmin, one, may, wish, maximize, signaltointerferenceplusnoise, ratio, sinr, legitimate, receiver, minimizing, eavesdropper, aim, fill, gap, maxfp, minfp, devising, unified, optimization, framework, main, results, threefold, first, extend, existing, maxfp, technique, called, quadratic, transform, minfp, develop, full, generalization, mixed, case, second, provide, minorizationmaximization, interpretation, proposed, unified, approach, thereby, establishing, convergence, also, obtaining, matrix, extension;, another, result, obtain, generalized, lagrangian, dual, transform, facilitates, solving, logarithmic, finally, present, three, typical, applicat


# Modeling

We will first form word2vec embeddings.
We form the document embedding by averaging over the word embeddings.

Finally, we can use the document embeddings to compute cosine similarity metrics and use it in K-means clustering.


## Word2Vec embedding

To form the Word2Vec embeddings, we will create \\( 10 \\)-dimensional vectors.

In [0]:
import mlflow

In [0]:
from pyspark.ml.feature import Word2Vec, Normalizer
from pyspark.sql.types import DoubleType

In [0]:
word2vec = Word2Vec(inputCol = 'clean_tokens', vectorSize = 10, outputCol = 'embedding')
with mlflow.start_run(run_name = 'run-1') as wordrun:
    example_df = df.select('clean_tokens')
    model = word2vec.fit(df)
    mlflow.spark.log_model(model, "word-model", signature=infer)

2023/12/21 09:43:38 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


2023/12/21 09:44:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/313095920668275/fdc5b08114db4979aa9d7751a587ad7e/artifacts/word-model/sparkml, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.
/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
df = model.transform(df)

In [0]:
from pyspark.ml.functions import vector_to_array

In [0]:
# Normalize the embedding vectors
normalizer = Normalizer(inputCol = 'embedding', outputCol = 'normalized_embedding', p = 2.0)
df = normalizer.transform(df)
df = df.withColumn('embedding_arr', vector_to_array('normalized_embedding'))

In [0]:
# Find instances where embedding_arr has length 0
# This not happen since we regularize the article embeddings to have length 1.
norm_udf = udf(lambda x: sum([el ** 2 for el in x]), DoubleType())
df = df.withColumn('embedding_norm', norm_udf('embedding_arr'))

In [0]:
# View papers with 0 embedding vector and remove them
eps = 1e-6
display(df.where(df.embedding_norm < eps).select('id', 'abstract'))
df = df.filter(df.embedding_norm > eps)

id,abstract



## Clustering

Now we have the document embeddings, we can now use them to compute cosine metric similarities.
Using similarities, we can cluster the documents.

The number of clusters must be specified beforehand, we will use \\(K = 8\\).

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
with mlflow.start_run(run_name = 'run-2'):
    K = 8
    kmeans = KMeans(k = K, distanceMeasure = "cosine", featuresCol = 'normalized_embedding', predictionCol = 'prediction', maxIter = 4)
    kmeans_model = kmeans.fit(df)
    mlflow.spark.log_model(kmeans_model, 'kmeans-model')

2023/12/21 09:50:58 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


2023/12/21 09:51:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/313095920668275/251d51e5b2034a40a48e583af581fc32/artifacts/kmeans-model/sparkml, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
df = kmeans_model.transform(df)


## PCA decomposition

To visualize the data in 2 dimensions we need to map the multidimensional embeddings into the plane. 

We can use _principal component analysis_ to achieve this. PCA forms linear combinations of the original axis, such that the variance of the dataset is preserved the most. 

In [0]:
from pyspark.ml.feature import PCA

In [0]:
with mlflow.start_run(run_name = 'run-3'):
    pca = PCA(k = 2, inputCol = 'normalized_embedding', outputCol = 'pca_features')
    pca_model = pca.fit(df)
    mlflow.spark.log_model(pca_model, 'pca-model')

2023/12/21 09:55:33 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


2023/12/21 09:56:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/313095920668275/078141c793ff4082b97af9d4afb738d4/artifacts/pca-model/sparkml, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
df = pca_model.transform(df)
df = df.withColumn('pca_features', vector_to_array('pca_features'))

In [0]:
# Save the result in Delta table
selectable_columns = ['id', 'authors_parsed', 'categories', 'submitter', 'update_date', 'title', 'abstract', 'embedding_arr', 'prediction', 'pca_features', 'year']
display(df.select(*selectable_columns).limit(3))

id,authors_parsed,categories,submitter,update_date,title,abstract,embedding_arr,prediction,pca_features,year
2305.02649,"List(List(Guo, Ke, ), List(Jing, Wei, ), List(Chen, Junbo, ), List(Pan, Jia, ))",cs.RO,Ke Guo,2023-05-05,CCIL: Context-conditioned imitation learning for urban driving,"Imitation learning holds great promise for addressing the complex task of autonomous urban driving, as experienced human drivers can navigate highly challenging scenarios with ease. While behavior cloning is a widely used imitation learning approach in autonomous driving due to its exemption from risky online interactions, it suffers from the covariate shift issue. To address this limitation, we propose a context-conditioned imitation learning approach that employs a policy to map the context state into the ego vehicle's future trajectory, rather than relying on the traditional formulation of both ego and context states to predict the ego action. Additionally, to reduce the implicit ego information in the coordinate system, we design an ego-perturbed goal-oriented coordinate system. The origin of this coordinate system is the ego vehicle's position plus a zero mean Gaussian perturbation, and the x-axis direction points towards its goal position. Our experiments on the real-world large-scale Lyft and nuPlan datasets show that our method significantly outperforms state-of-the-art approaches.","List(0.26779809606130345, -0.021201288226136154, 0.059815622754490295, -0.33218383169904886, 0.22754517692028842, 0.09961459858577139, -0.290823463809687, -0.43780028804579235, 0.01846008760990836, 0.6896540063066591)",4,"List(-0.06838093397805217, -0.26521568090144404)",2023
2305.02677,"List(List(Wang, Teng, ), List(Zhang, Jinrui, ), List(Fei, Junjie, ), List(Zheng, Hao, ), List(Tang, Yunlong, ), List(Li, Zhe, ), List(Gao, Mingqi, ), List(Zhao, Shanshan, ))",cs.CV,Teng Wang,2023-07-07,Caption Anything: Interactive Image Description with Diverse Multimodal Controls,"Controllable image captioning is an emerging multimodal topic that aims to describe the image with natural language following human purpose, $\textit{e.g.}$, looking at the specified regions or telling in a particular text style. State-of-the-art methods are trained on annotated pairs of input controls and output captions. However, the scarcity of such well-annotated multimodal data largely limits their usability and scalability for interactive AI systems. Leveraging unimodal instruction-following foundation models is a promising alternative that benefits from broader sources of data. In this paper, we present Caption AnyThing (CAT), a foundation model augmented image captioning framework supporting a wide range of multimodel controls: 1) visual controls, including points, boxes, and trajectories; 2) language controls, such as sentiment, length, language, and factuality. Powered by Segment Anything Model (SAM) and ChatGPT, we unify the visual and language prompts into a modularized framework, enabling the flexible combination between different controls. Extensive case studies demonstrate the user intention alignment capabilities of our framework, shedding light on effective user interaction modeling in vision-language applications. Our code is publicly available at https://github.com/ttengwang/Caption-Anything.","List(0.38816387190649027, -0.2384371930654559, 0.4529631938748501, -0.46817670581690235, 0.22739352085421488, 0.28351875146127364, -0.11113484039391547, -0.15463716094418192, 0.24044627179566155, 0.37675286457229534)",2,"List(-0.06990281570148421, -0.5280882523295725)",2023
2305.02690,"List(List(Cheon, Gi-Sang, ), List(Kang, Bumtle, ), List(Kim, Suh-Ryung, ), List(Ryu, Homoon, ))",math.CO,Bumtle Kang,2023-05-05,Row graphs of Toeplitz matrices,"In this paper, we study row graphs of Toeplitz matrices. The notion of row graphs was introduced by Greenberg et al. in 1984 and is closely related to the notion of competition graphs, which has been extensively studied since Cohen had introduce

In [0]:
df.select(*selectable_columns).write.mode('overwrite').option('mergeSchema', 'true').format('delta').saveAsTable('main.default.ARXIV_ANALYSIS')


## Registering model

Now we have created the relevant models and stored the inference in a table. We can do the visualization now in a separate notebook.

However, we should build a pipeline of the steps made in order to save and register the total model.

In [0]:
from pyspark.ml import Pipeline 
from mlflow.models.signature import infer_signature

In [0]:
word2vec = Word2Vec(inputCol = 'clean_tokens', vectorSize = 10, outputCol = 'embedding')
normalizer = Normalizer(inputCol = 'embedding', outputCol = 'normalized_embedding', p = 2.0)

K = 8
kmeans = KMeans(k = K, distanceMeasure = "cosine", featuresCol = 'normalized_embedding', predictionCol = 'prediction', maxIter = 4)
pca = PCA(k = 2, inputCol = 'normalized_embedding', outputCol = 'pca_features')

pipeline = Pipeline(stages=[word_tokenizer, stop_words_remover, word2vec, normalizer, kmeans, pca])

In [0]:
with mlflow.start_run(run_name = 'pipeline-run-1'):
    train_features = df.select('abstract_math_free')
    pipeline_model = pipeline.fit(train_features)
    predictions = pipeline_model.transform(train_features)
    output_signature = predictions.select('abstract_math_free', 'prediction')
    signature = infer_signature(train_features, output_signature)
    mlflow.spark.log_model(pipeline_model, 'arxiv-model', signature = signature)

/databricks/python/lib/python3.10/site-packages/mlflow/models/signature.py:213: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  outputs = _infer_schema(model_output) if model_output is not None else None
2023/12/21 10:48:46 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed

2023/12/21 10:48:49 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
2023/12/21 10:49:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/313095920668275/01ca3e231e4a4cc39b1dc769f51a7659/artifacts/arxiv-model/sparkml, flavor: spark), fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
catalog = "main"
schema = "default"
model_name = "axiv_classifier"
mlflow.set_registry_uri("databricks-uc")
mlflow.register_model(
    model_uri="runs:/01ca3e231e4a4cc39b1dc769f51a7659/arxiv-model",
    name=f"{catalog}.{schema}.{model_name}"
)

Successfully registered model 'main.default.axiv_classifier'.


2023/12/21 10:50:18 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Uploading artifacts:   0%|          | 0/33 [00:00<?, ?it/s]

2023/12/21 10:50:23 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Created version '1' of model 'main.default.axiv_classifier'.


<ModelVersion: aliases=[], creation_timestamp=1703155818363, current_stage=None, description='', last_updated_timestamp=1703155825660, name='main.default.axiv_classifier', run_id='01ca3e231e4a4cc39b1dc769f51a7659', run_link=None, source='dbfs:/databricks/mlflow-tracking/313095920668275/01ca3e231e4a4cc39b1dc769f51a7659/artifacts/arxiv-model', status='READY', status_message='', tags={}, user_id='tuomas.myllymaki98@outlook.com', version='1'>


# Endpoint querying

Now after registering the models and creating and endpoint, we can query it. After querying, we should get a prediction of the cluster.

In [0]:
import os, requests, json
import pandas as pd

In [0]:
headers = {'Authorization': f'Bearer {secret_key}', 'Content-Type': 'application/json'}
dataset = pd.DataFrame({
    "abstract_math_free": ["This is a test. We can consider calculating the mass of the sun by using radio spectrotrometry."]
})
ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
data_json = json.dumps(ds_dict, allow_nan=True)
response = requests.request(method='POST', headers=headers, url=url, data=data_json)
if response.status_code != 200:
    raise Exception(f'Request failed with status {response.status_code}, {response.text}')

In [0]:
response_json = json.loads(response.text)
predictions = response_json['predictions']
predictions

[5]


So our model thinks the example is in group 5.

We can try and compare our test abstract to other abstracts in the group 5. 

In [0]:
similar_df = spark.sql('select id, abstract from main.default.ARXIV_ANALYSIS where prediction == 5')
display(similar_df.limit(10))

id,abstract
2207.03898,"Statistical classification of the Helios solar wind observations into several populations sorted by bulk speed has revealed an outward acceleration of the wind. The faster the wind is, the smaller is this acceleration in the 0.3 - 1 au radial range (Maksimovic et al. 2020). In this article we show that recent measurements from the Parker Solar Probe (PSP) are compatible with an extension closer to the Sun of the latter Helios classification. For instance the well established bulk speed/proton temperature (u,Tp) correlation and bulk speed/electron temperature (u,Te) anti-correlation, together with the acceleration of the slowest winds, are verified in PSP data. We also model the combined PSP and Helios data, using empirical Parker-like models for which the solar wind undergoes an ""iso-poly"" expansion: isothermal in the corona, then polytropic at distances larger than the sonic point radius. The polytropic indices are derived from the observed temperature and density gradients. Our modelling reveals that the electron thermal pressure has a major contribution in the acceleration process of slow and intermediate winds (in the range of 300-500 km/s at 1 au), over a broad range of distances and that the global (electron and protons) thermal energy, alone, is able to explain the acceleration profiles. Moreover, we show that the very slow solar wind requires in addition to the observed pressure gradients, another source of acceleration."
2207.03903,"We report the discovery of five cyano derivatives of propene towards TMC-1 with the QUIJOTE line survey: $trans$ and $cis$-crotononitrile ($t$-CH$_3$CHCHCN, $c$-CH$_3$CHCHCN), methacrylonitrile (CH$_2$C(CH$_3$)CN), and $gauche$ and $cis$-allyl cyanide ($g$-CH$_2$CHCH$_2$CN and $c$-CH$_2$CHCH$_2$CN). The observed transitions allowed us to derive a common rotational temperature of 7$\pm$1 K for all them. The derived column densities are N($t$-CH$_3$CHCHCN)=(5$\pm$0.5)$\times$10$^{10}$ cm$^{-2}$, N($c$-CH$_3$CHCHCN)=(1.3$\pm$0.2)$\times$10$^{11}$ cm$^{-2}$, N(CH$_2$C(CH$_3$)CN)=(1.0$\pm$0.1)$\times$10$^{11}$ cm$^{-2}$, N($g$-CH$_2$CHCH$_2$CN)=(8.0$\pm$0.8)$\times$10$^{10}$ cm$^{-2}$, and N($c$-CH$_2$CHCH$_2$CN)=(7.0$\pm$0.7)$\times$10$^{10}$ cm$^{-2}$, respectively. The abundance of cyano-propene relative to that of propene is thus $\sim$10$^{-2}$, which is considerably lower than those of other cyano derivatives of abundant hydrocarbons. Upper limits are obtained for two ethynyl derivatives of propene ($E$ and $Z$-CH$_3$CHCHCCH)."
2207.03968,"We analyze the galaxy pairs in a set of volume limited samples from the SDSS to study the effects of minor interactions on the star formation rate (SFR) and colour of galaxies. We carefully design control samples of the isolated galaxies by matching the stellar mass and redshift of the minor pairs. The SFR distributions and colour distributions in the minor pairs differ from their controls at $>99\%$ significance level. We also simultaneously match the control galaxies in stellar mass, redshift and local density to assess the role of the environment. The null hypothesis can be rejected at $>99\%$ confidence level even after matching the environment. Our analysis shows a quenching in the minor pairs where the degree of quenching decreases with the increasing pair separation and plateaus beyond 50 kpc. We also prepare a sample of minor pairs with $H_{\alpha}$ line information. We calculate the SFR of these galaxies using the $H_{\alpha}$ line and repeat our analysis. We observe a quenching in the $H_{\alpha}$ sample too. We find that the majority of the minor pairs are quiescent systems that could be quenched due to minor interactions. Combining data from the Galaxy Zoo and Galaxy Zoo2, we find that only $\sim 1\%$ galaxies have a dominant bulge, $4\%-7\%$ galaxies host a bar, and $5\%-10\%$ galaxies show the AGN activity in minor pairs. This indicates that the presence of bulge, bar and AGN activity plays an insignificant role i


Looking at the results, we can see that the test abstract does fit into this category quite well. There's references to astronomical phenomena and astrophysics.

However, there are some chemistry papers also in the same category 5. So the categories are not perfectly separated.